Name: Prasanna Hegde  
Matr. 422985 

In [ ]:
from bs4 import BeautifulSoup
import random
import re       #regular expressions lib

-Load and parse the TMX file.  
-The TMX file has contents in html format. To parse it, we will use BeautifulSoup package  
-The file has several translational pairs/units(tu)  

The opensubtitles dataset is used here. https://opus.nlpl.eu/OpenSubtitles/de&fr/v2024/OpenSubtitles

In [ ]:


#tmx_file = r"dataset\de-fr_04-11-2015_Website_final_stripped.tmx"
tmx_file = r"dataset\de-fr_opensub.tmx"
with open(tmx_file, 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'xml')

We will fetch all translation pairs and them to a list.

In [10]:
translation_pairs = []
for tu in soup.find_all('tu'):
    source_text = tu.find_all('tuv')[0].seg.get_text()
    target_text = tu.find_all('tuv')[1].seg.get_text()
    translation_pairs.append((source_text, target_text))

In [11]:
print('Total translational pairs=', len(translation_pairs))
print('sample translational pairs',translation_pairs[0:5]) 

Total translational pairs= 42020
sample translational pairs [('LOS ANGELES IM JAHR 2029 A. D.', 'LOS ANGELES 2029 APRES J. - C.'), ('DIE MASCHINEN ERSTANDEN AUS DER ASCHE DES NUKLEAREN FEUERS AUF.', "LES MACHINES ONT SURGI DES CENDRES DE L'ENFER NUCLEAIRE."), ('IHR VERNICHTUNGSKRIEG GEGEN DIE MENSCHHEIT WÜTETE SEIT LANGEM,', "ELLES ESSAYAIENT D'EXTERMINER L'HOMME DEPUIS DES DECENNIES,"), ('ABER DIE LETZTE SCHLACHT SOLLTE NICHT IN DER ZUKUNFT STATTFINDEN.', "MAIS LA BATAILLE FINALE N'AURAIT PAS LIEU DANS LE FUTUR."), ('SONDERN HIER, IN UNSERER GEGENWART.', 'ELLE AURAIT LIEU ICI, DE NOS JOURS.')]


We will select translations which have 5 words or more. 

In [ ]:

def clean_text(text):
    """_summary_# Function that removes only leading non-word characters

    Sometimes, the subtitles could have '-' in the starting

    """
    return re.sub(r'^[^\w]+', '', text.strip())  

In [ ]:
random.seed(42)


# Filter translation pairs (5+ words in the source text after cleaning)
filtered_pairs = [
    (clean_text(pair[0]), clean_text(pair[1]))  # Clean only the source and target text
    for pair in translation_pairs 
    if len(clean_text(pair[0]).split()) > 5
]

print(len(filtered_pairs))

selected_pairs = random.sample(filtered_pairs, 1000)  # select 1000 pairs from the filtered pairs

print(selected_pairs)

if len(selected_pairs) == len(set(selected_pairs)):
    print("No duplicates found in the selected samples")


15496
[('Aber jetzt sind wir an der Reihe, sein weises Wesen gegen die giftigen Pfeile des Lebens zu schützen.', 'Mais sa sérénité exige de nous un rempart contre les flèches de ce monde.'), ('Aber das hier könnte sie belasten.', 'Mais cela va la compromettre.'), ('Aufruhr entstand letzte Nacht in einigen Elektronikläden, die ausverkauft waren.', "Hier, c'était l'émeute dans les magasins en rupture de stock."), ('Ich wollte dem Richter einen Rolls Royce schenken!', "J'arrivais pas à y croire, je devrais offrir à ce juge une Rolls Royce!"), ('Halten wir es mal zehn Minuten lang sauber?"', 'C\'est possible de ne rien salir pendant 10 min?"'), ('Er hat nicht nur die Körper gegessen.', "Ils ont inspecté l'intérieur."), ('Sie hat meine Mutter damit fast zu Tode geängstigt.', 'Elle terrorisait ma mère, à la faire mourir.'), ('Oh ja, die ist wirklich süß.', 'Oui, elle est très gentille.'), ('Na, dann wollen wir uns alle mal hinsetzen!', 'Non, non ça va.'), ('Zum Glück haben Sie den nicht gehe

We have 15496 translations which have atleast 5 words in them. From these, we selected randomly 1000 pairs. 

In [14]:
import csv

# Save selected translation pairs to a CSV file
csv_file = r'deliverables\df_csv\Dataset_A.csv'

with open(csv_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['German', 'French'])  # Add header
    writer.writerows(selected_pairs)

print(f"Saved {len(selected_pairs)} translation pairs to {csv_file}.")

Saved 1000 translation pairs to deliverables\df_csv\Dataset_A.csv.


We split the Dataset A into training and test set. The split is chosen as 80-20 for train and test respectively. 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load CSV as a DataFrame
df = pd.read_csv(r'deliverables\df_csv\Dataset_A.csv')

# Split into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the splits to new CSV files
train_df.to_csv(r'deliverables\df_csv\train_A.csv', index=False)
test_df.to_csv(r'deliverables\df_csv\test_A.csv', index=False)

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

Training set size: 800
Test set size: 200
